In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
physical_devices = tf.config.list_physical_devices('CPU')
print("Available GPUs:", physical_devices)

Available GPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    r'C:\Users\miral\Assigment-1\LS-ML-week1\images', 
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(64, 64),
    shuffle=True,
    seed=12,
    validation_split=0.1,
    subset='training'
)
class_names = ds_train.class_names
print("Class names:", class_names)
AUTOTUNE = tf.data.AUTOTUNE
print(class_names)

Found 269 files belonging to 2 classes.
Using 243 files for training.
Class names: ['Bart', 'Homer']
['Bart', 'Homer']


In [3]:
ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    r'C:\Users\miral\Assigment-1\LS-ML-week1\images', 
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(64, 64),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='validation'
)


Found 269 files belonging to 2 classes.
Using 26 files for validation.


In [4]:
print(ds_train)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential([
    layers.Flatten(input_shape=(64,64,3)),
    Dense(264, activation='relu',kernel_regularizer='l2'),
    Dense(128, activation='relu',kernel_regularizer='l2'),
    Dense(64,activation='relu',kernel_regularizer='l2'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


c:\Users\miral\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', 
                               mode='min',          
                               patience=5,           
                               verbose=1)        


def normalize_img(image, label):
   
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


ds_train_normalized = ds_train.map(normalize_img)
ds_validation_normalized = ds_validation.map(normalize_img)

model.fit(ds_train_normalized,
          epochs=100,
          validation_data=ds_validation_normalized,
          callbacks=[early_stopping])



Epoch 1/100


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.5390 - loss: 10.1525 - val_accuracy: 0.3462 - val_loss: 7.7784
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.5303 - loss: 6.9809 - val_accuracy: 0.7692 - val_loss: 5.8139
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.6954 - loss: 5.6868 - val_accuracy: 0.7308 - val_loss: 5.0482
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.7833 - loss: 4.9392 - val_accuracy: 0.3462 - val_loss: 5.1743
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - accuracy: 0.6016 - loss: 4.7162 - val_accuracy: 0.7308 - val_loss: 4.0775
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.7223 - loss: 4.0428 - val_accuracy: 0.8846 - val_loss: 3.5665
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.8273 - loss: 3.5459 - val_accuracy: 0.9615 - val_loss: 3.2022
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.8378 - loss: 3.2329 - val_accuracy: 0.8846 - val_loss: 2.9718
Ep

In [23]:
# Evaluate the model on the validation dataset
test_loss, test_accuracy = model.evaluate(ds_validation, verbose=0)

# Convert accuracy to percentage format
accuracy_percentage = test_accuracy * 100

# Print accuracy in percentage format
print(f"Accuracy: {accuracy_percentage:.2f}%")


Accuracy: 96.15%


In [24]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(64, 64))  # Load and resize image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the input shape
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

def predict_image(img_path, model):
    img_array = load_and_preprocess_image(img_path)  # Load and preprocess image
    prediction = model.predict(img_array)  # Make prediction
    predicted_prob = prediction[0][0]  # Get the probability of the positive class
    predicted_class = int(predicted_prob > 0.5)  # Classify as 1 if probability > 0.5, else 0
    return predicted_class, predicted_prob

# Assuming 'model' is your trained model
# model = tf.keras.models.load_model('path_to_your_model.h5')

img_path = "images/Bart/bart3.bmp"  # Replace with the path to your test image
predicted_class, predicted_prob = predict_image(img_path, model)

print(f"Predicted class: {predicted_class}")
print(f"Probability: {predicted_prob:.2f}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Predicted class: 0
Probability: 0.04


In [27]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(64, 64))  # Load and resize image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the input shape
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

def predict_image(img_path, model):
    img_array = load_and_preprocess_image(img_path)  # Load and preprocess image
    prediction = model.predict(img_array)  # Make prediction
    predicted_prob = prediction[0][0]  # Get the probability of the positive class
    predicted_class = int(predicted_prob > 0.5)  # Classify as 1 if probability > 0.5, else 0
    return predicted_class, predicted_prob

# Assuming 'model' is your trained model
# model = tf.keras.models.load_model('path_to_your_model.h5')

img_path = "images/Homer/homer7.bmp"  # Replace with the path to your test image
predicted_class, predicted_prob = predict_image(img_path, model)

print(f"Predicted class: {predicted_class}")
print(f"Probability: {predicted_prob:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted class: 1
Probability: 0.98
